In [1]:
import requests
from bs4 import BeautifulSoup as BS
import pandas as pd
import re
from tqdm.notebook import tqdm
from time import sleep
import plotly.express as px


ModuleNotFoundError: No module named 'bs4'

In [ ]:
URL = 'https://www.opensecrets.org/races/summary?cycle=2020&id=TN02&spec=N'

res = requests.get(URL)

In [ ]:
res.status_code

In [ ]:
soup = BS(res.text)

In [ ]:
print(soup.prettify())

In [ ]:
type(soup.find_all('table', attrs = {'class':'DataTable'})[0])

In [ ]:
html_df = str(soup.find_all('table', attrs = {'class':'DataTable'})[0])

In [ ]:
district02_df = pd.read_html(html_df)

In [ ]:
district02_df = district02_df[0]

In [ ]:
district02_df['Candidate'] = district02_df['Candidate'].str.replace("•","")
district02_df

In [ ]:
district02_df[['Candidate','party', 'incumbent', 'winner']] = district02_df.Candidate.str.extract(r'(\w+\s\w+\s)(\W\w\W\s?)(\s?\W?\w*)(\s?\W?\w*)' , expand = True) 

In [ ]:
district02_df = district02_df.replace('','no')
district02_df = district02_df.replace(['(',')'], '')

In [ ]:
district02_df

In [ ]:
soup.find_all('li', {'class':'SubNav-item'})

In [ ]:
#https://data36.com/scrape-multiple-web-pages-beautiful-soup-tutorial/
n = 1
all_districts_tn=pd.DataFrame()
for n in range(1,10):
    URL = f'https://www.opensecrets.org/races/summary?cycle=2020&id=TN0{n}&spec=N'
    res1 = requests.get(URL)
    soup1 = BS(res1.text)
    tn_districts = pd.read_html(str(soup1.find_all('table', attrs = {'class':'DataTable'})[0]))[0]
    tn_districts['districts'] = n
    tn_districts['state'] = 'TN'
    all_districts_tn = all_districts_tn.append(tn_districts)
    n+=1
tn_districts

In [ ]:
all_districts_tn

In [ ]:
all_districts_tn.Candidate.to_list()

In [ ]:
all_districts_tn[['name', 'party', 'nan', 'incumbent', 'winner']] = all_districts_tn.Candidate.str.extract(r'(\W?\w+?\s?\w+\s\w+\s)(\W\w\W\s?)(\s?\W?\w*)(\s?\W?\w*\s?\W?)?(\s?\w+)?' , expand = True) 

In [ ]:
all_districts_tn['incumbent'].to_list()

In [ ]:
all_districts_tn = (all_districts_tn.drop(columns = ['nan'])
                    .fillna(False).
                    replace({' Winner': True , ' Incumbent •': True, '':False}))

In [ ]:
all_districts_tn.head()

In [ ]:
#getting all data from all states:
states = ["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DE", "FL", "GA", 
          "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", 
          "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", 
          "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC", 
          "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"]
url2 = 'https://www.britannica.com/topic/United-States-House-of-Representatives-Seats-by-State-1787120'
r = requests.get(url2)
soup2 = BS(r.text)
r

In [ ]:
num_of_dist = pd.read_html(str(soup2.find_all('table')))

In [ ]:
num_of_dist = num_of_dist[0]
num_of_dist.info()

In [ ]:
num_of_dist = num_of_dist.drop(index = 50)

In [ ]:
num_of_dist['abbrev'] = states

In [ ]:
num_of_dist

In [ ]:
representatives = num_of_dist['representatives'].to_list()
state_abbrev = num_of_dist['abbrev'].to_list()

In [ ]:
num_of_dist['representatives'].sum()

In [ ]:
reps_by_state = list(zip(state_abbrev,representatives))

In [ ]:
reps_by_state

In [ ]:
#i = 10
all_districts_us_10= pd.DataFrame()
all_districts_us_9 = pd.DataFrame()
for state in states:
    for i in range(1,54):
        sleep(0.1)
        if i < 10:
            try:
                URL2 = f'https://www.opensecrets.org/races/summary?cycle=2020&id={state}0{i}&spec=N'
                res2 = requests.get(URL2)
                soup3 = BS(res2.text)
                print(URL2,res2)
                us_districts1 = pd.read_html(str(soup3.find_all('table', attrs = {'class':'DataTable'})[0]))[0]
                us_districts1['state'] = state
                us_districts1['district'] = i
                all_districts_us_9 = all_districts_us_9.append(us_districts1)
            except IndexError:
                break
        if i >= 10:
            try:
                URL3 = f'https://www.opensecrets.org/races/summary?cycle=2020&id={state}{i}&spec=N'
                res3 = requests.get(URL3)
                soup4 = BS(res3.text)
                print(URL3, res3)
                us_districts2 = pd.read_html(str(soup4.find_all('table', attrs = {'class':'DataTable'})[0]))[0]
                us_districts2['state'] = state
                us_districts2['district'] = i
                all_districts_us_10 = all_districts_us_10.append(us_districts2)
            except IndexError:
                break
    i+=1


In [ ]:
frames = [all_districts_us_9, all_districts_us_10]
all_districts_us = pd.concat(frames)

In [ ]:
all_districts_us

In [ ]:
all_districts_us[['name', 'party', 'nan', 'incumbent', 'winner']] = all_districts_us.Candidate.str.extract(r'(\W\w+?\W?\w+?\s?\w+\s\w+)\W?\s(\W\w\W\s?)(\s?\W?\w*)(\s?\W?\w*\s?\W?)?(\s?\w+)?' , expand = True)

In [ ]:
all_districts_us

In [ ]:
all_districts_us = all_districts_us.reset_index()

In [ ]:
all_districts_us

In [ ]:
all_districts_us['winner']= all_districts_us[all_districts_us['index'] == 0]['winner'].fillna(1)

In [ ]:
all_districts_us['incumbent'] = all_districts_us['incumbent'].replace({'Winner': 0})

In [ ]:
all_districts_us.winner.to_list()

In [ ]:
all_districts_us = (all_districts_us.drop(columns = ['nan'])
                    .fillna(False)
                    .replace({ ' Winner': 1 , 'Incumbent •': 1, '': 0}))

In [ ]:
all_districts_us

In [ ]:
all_districts_us['incumbent'] = all_districts_us['incumbent'].astype(bool)
all_districts_us['winner'] = all_districts_us['winner'].astype(bool)
all_districts_us.info()

In [ ]:
x = 1
df = [0]
for x in range(1,50):
    df.append(all_districts_us[all_districts_us['state'] == states[x]]['winner'].astype(bool).sum())
len(df)

In [ ]:
num_of_dist['winners'] = df
num_of_dist

In [ ]:

#Finding and correcting districts with wrong amount of winners
num_of_dist[num_of_dist['winners']!=num_of_dist['representatives']]

In [ ]:
ca_districts = all_districts_us[all_districts_us['state'] == 'MA']
ca_districts[ca_districts['winner']==True]

In [ ]:
#Drop IL district 19 index 730
#Drop IA district 5 index 187
#Drop LA district 7 index 224
#MA seems ok
#Drop MI dist 15 index 746
#Drop MO dist 9 index 315
#Drop NJ dist 13 index 753
#Drop NY dist 28 index 794
#Drop OH dist 17 index 821
#Drop PA dist 19 index 843
all_districts_us = all_districts_us.drop(index = [730,187,224,746,315,753,794,821,843])

In [ ]:
#Now correct when including special elections
all_districts_us['winner'].astype(bool).sum()

In [ ]:
all_districts_us.to_csv('all_districts_us')